In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Multiply, concatenate
from keras.layers import Attention, BatchNormalization
from keras.callbacks import EarlyStopping
from statsmodels.tsa.arima.model import ARIMA
import pywt

# --- Step 1: Generate Synthetic Wind Speed Data ---
np.random.seed(42)
time_index = pd.date_range("2023-01-01", periods=1000, freq="H")
base_wind = 10 + 5 * np.sin(np.linspace(0, 20*np.pi, 1000))  # Base pattern
noise = 3 * np.random.randn(1000)  # Turbulence
synthetic_wind = np.maximum(0, base_wind + noise)  # Non-negative

# Create DataFrame with datetime features
df = pd.DataFrame({
    'timestamp': time_index,
    'wind_speed': synthetic_wind,
    'hour': time_index.hour,
    'day_of_week': time_index.dayofweek,
    'month': time_index.month
}).set_index('timestamp')

# --- Step 2: Wavelet Decomposition ---
def wavelet_decompose(data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    return np.concatenate(coeffs)

df['wind_wavelet'] = wavelet_decompose(df['wind_speed'])[:len(df)]

# --- Step 3: Feature Engineering ---
df['rolling_avg_6h'] = df['wind_speed'].rolling(6).mean()
df['rolling_std_12h'] = df['wind_speed'].rolling(12).std()
df.fillna(0, inplace=True)

# --- Step 4: Train-Test Split ---
train_size = int(0.8 * len(df))
train, test = df.iloc[:train_size], df.iloc[train_size:]

# --- Step 5: Data Scaling ---
scaler = MinMaxScaler()
cols_to_scale = ['wind_speed', 'wind_wavelet', 'rolling_avg_6h', 'rolling_std_12h']
train_scaled = scaler.fit_transform(train[cols_to_scale])
test_scaled = scaler.transform(test[cols_to_scale])

# --- Step 6: Create Sequences ---
def create_sequences(data, target_col=0, n_steps=24):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps, :])
        y.append(data[i+n_steps, target_col])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled)
X_test, y_test = create_sequences(test_scaled)

# --- Step 7: Build Hybrid LSTM-Attention Model ---
def build_hybrid_model(input_shape):
    # Input Layer
    inputs = Input(shape=input_shape)
    
    # LSTM Branch
    lstm_out = LSTM(64, return_sequences=True)(inputs)
    lstm_out = BatchNormalization()(lstm_out)
    
    # Attention Mechanism
    attention = Attention()([lstm_out, lstm_out])
    attention = Multiply()([lstm_out, attention])
    
    # Second LSTM
    lstm_out2 = LSTM(32)(attention)
    
    # Dense Layers
    dense_out = Dense(32, activation='relu')(lstm_out2)
    dense_out = Dropout(0.2)(dense_out)
    
    # Output
    output = Dense(1)(dense_out)
    
    return Model(inputs=inputs, outputs=output)

model = build_hybrid_model((X_train.shape[1], X_train.shape[2]))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# --- Step 8: Train Model ---
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

C:\Users\fardo\AppData\Local\Temp\ipykernel_19880\3147430847.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range("2023-01-01", periods=1000, freq="H")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 24, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 24, 64)    │     17,664 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 24, 64)    │        256 │ lstm[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 24, 64)    │          0 │ batch_normalizat… │
│ (Attention)         │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 24, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │     12,416 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      1,056 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         33 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 31,425 (122.75 KB)

 Trainable params: 31,297 (122.25 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - loss: 0.0682 - mae: 0.2096 - val_loss: 0.1252 - val_mae: 0.3123
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0369 - mae: 0.1508 - val_loss: 0.1151 - val_mae: 0.2970
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0256 - mae: 0.1271 - val_loss: 0.1096 - val_mae: 0.2886
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0217 - mae: 0.1158 - val_loss: 0.1053 - val_mae: 0.2820
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0225 - mae: 0.1188 - val_loss: 0.1026 - val_mae: 0.2780
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0219 - mae: 0.1176 - val_loss: 0.0960 - val_mae: 0.2675
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0234 - mae: 0.1207 - val_loss: 0.0948 - val_mae: 0.2652
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0202 - mae: 0.1128 - val_loss: 0.0907 - val_mae: 0.2587
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0

In [2]:
# --- Step 9: Evaluate & Forecast ---
y_pred = model.predict(X_test)

# Inverse scaling
y_test_inv = scaler.inverse_transform(
    np.concatenate([y_test.reshape(-1,1), np.zeros((len(y_test), len(cols_to_scale)-1))], axis=1)
)[:,0]
y_pred_inv = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((len(y_pred), len(cols_to_scale)-1))], axis=1)
)[:,0]

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step


In [ ]:
# --- Step 10: Plot Results ---
plt.figure(figsize=(14, 6))
plt.plot(test.index[len(test.index)-len(y_test):], y_test_inv, label='Actual', color='blue')
plt.plot(test.index[len(test.index)-len(y_test):], y_pred_inv, label='Predicted', color='red', linestyle='--')
plt.title('Hybrid LSTM Wind Speed Forecast')
plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.legend()
plt.grid()
plt.savefig('hybrid_lstm_forecast.png')
plt.show()

# --- Step 11: Error Metrics ---
mse = np.mean((y_test_inv - y_pred_inv)**2)
print(f"\nMSE: {mse:.2f}")
print(f"MAE: {np.mean(np.abs(y_test_inv - y_pred_inv)):.2f}")

# --- Step 12: Save Model ---
model.save('hybrid_lstm_wind_forecast.h5')